In [11]:
import os
import yfinance as yf
import pandas as pd
import streamlit as st
from datetime import datetime, timedelta

import yfinance as yf
import pandas as pd
import numpy as np
import streamlit as st
import os
from datetime import datetime, timedelta
from typing import List

In [12]:
if not os.path.exists('./data'):
    os.makedirs('./data')

def download_stock_data(symbols: List[str], target_symbols: List[str], start_date: str, end_date: str):
    all_symbols = list(symbols + target_symbols)
    data = pd.DataFrame()
    remove_symbols = []
    for symbol in list(set(all_symbols)):
        file_name = f"./data/{symbol}.csv"
        if os.path.isfile(file_name):
            stock_data = pd.read_csv(file_name, index_col=0, parse_dates=True)
            stock_data.index = pd.to_datetime(stock_data.index)
            last_date = stock_data.index[-1]
            if last_date >= pd.Timestamp(datetime.now()):
                new_data = yf.download(symbol, start=last_date.date() + timedelta(days=1), end=end_date, interval='1d')
                if not new_data.empty and pd.isna(new_data['Adj Close'].iloc[-1]):
                    stock_data = stock_data.append(new_data)
                    stock_data = stock_data[~stock_data.index.duplicated(keep='last')]
                    with open(file_name, mode='w') as f:
                        stock_data.to_csv(f)
                else:
                    remove_symbols.append(symbol)
            else:
                pass
        else:
            stock_data = yf.download(symbol, start=start_date, end=end_date, interval='1d')
            if not stock_data.empty and pd.isna(stock_data['Adj Close'].iloc[-1]):
                with open(file_name, mode='w') as f:
                    stock_data.to_csv(f)
            else:
                remove_symbols.append(symbol)
        try:
            data[symbol] = stock_data['Adj Close']
        except Exception as e:
            remove_symbols.append(symbol)
            # print(symbol, e)
            pass
    for item in list(set(remove_symbols)):
        if item in symbols:
            symbols.remove(item)
        if item in target_symbols:
            target_symbols.remove(item)
    with open("./feat_symbols.txt", "w") as f:
        for symbol in symbols:
            f.write(f"{symbol}\n")
    with open("./target_symbols.txt", "w") as f:
        for symbol in target_symbols:
            f.write(f"{symbol}\n")

    return data, symbols, target_symbols

def prepare_data(data, symbols, target_symbols, shift=1):
    prepared_data = {}
    for target_symbol in target_symbols:
        stock_data = pd.DataFrame(data[target_symbol])
        for symbol in symbols:
            if symbol != target_symbol:
                stock_data[symbol] = data[symbol]
        prepared_data[target_symbol] = stock_data
    
    for key in prepared_data.keys():
        # shiftの先頭行を排除
        prepared_data[key] = prepared_data[key].iloc[shift:,:]

        # NoneをNaNに置換
        prepared_data[key] = prepared_data[key].replace(to_replace=['None', 'null', 'nan', 'NA'], value=np.nan)

        # 欠損値NaNを過去最新の値で埋める
        prepared_data[key] = prepared_data[key].fillna(method="ffill")
        prepared_data[key] = prepared_data[key].dropna(axis=1)            

        if prepared_data[key].isnull().any().any():
            st.write(f"{key} : NaN values found.")
    
    return prepared_data

In [13]:
# 銘柄リストを取得する関数
def get_symbol_list(file):
    with open(file, "r") as f:
        symbols = [line.strip() for line in f.readlines()]
    return symbols

In [14]:
def get_prepared_data(symbols, target_symbols, years, shift):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=years*365)
    data, symbols, target_symbols = download_stock_data(symbols, target_symbols, start_date, end_date)
    prepared_data = prepare_data(data, symbols, target_symbols, shift=shift)

    return prepared_data

In [15]:
years = 5 # 学習データの期間
shift = 5 # n 期先予測

symbols = get_symbol_list("feat_symbols.txt")
target_symbols = get_symbol_list("target_symbols.txt")

dict_prepared_data = get_prepared_data(symbols, target_symbols, years, shift)

C:\Users\rodin\AppData\Local\Temp\ipykernel_18340\2969299210.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[symbol] = stock_data['Adj Close']
C:\Users\rodin\AppData\Local\Temp\ipykernel_18340\2969299210.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[symbol] = stock_data['Adj Close']
C:\Users\rodin\AppData\Local\Temp\ipykernel_18340\2969299210.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

In [16]:
import pickle

with open("tests/prepared_data.pkl", "wb") as tf:
    pickle.dump(dict_prepared_data,tf)

with open("tests/prepared_data.pkl", "rb") as tf:
    dict_prepared_data = pickle.load(tf)
    
for key in dict_prepared_data.keys():
    display(dict_prepared_data[key])

,4584.T,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,1925.T,1928.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-01,1129.0,1181.539185,2578.757080,5039.168457,1047.106689,984.559326,1365.645874,1761.031006,3409.902344,1658.666992,...,131.141342,19.050804,181.230804,40.532646,25.731714,66.262154,79.292213,109.272720,47.827251,80.757622
2018-05-02,1153.0,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,129.728363,19.190237,180.456192,40.437065,23.412752,64.859131,73.398384,109.749229,47.948879,77.368500
2018-05-03,1153.0,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,127.625664,19.153540,181.013535,40.384930,22.560074,65.565346,75.658890,107.618919,47.305962,77.833290
2018-05-04,1153.0,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,128.937729,19.219591,181.759827,40.306732,22.615856,65.989082,75.439262,108.590645,48.183456,78.975906
2018-05-07,1155.5,1162.568604,2546.771240,5039.168457,1042.035889,967.918762,1354.645508,1720.817017,3436.165283,1651.217651,...,129.021820,19.278297,181.287476,40.080833,22.679607,65.932564,76.061584,109.310104,48.226902,79.547218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-17,249.0,1503.000000,2478.000000,4245.000000,1030.000000,780.000000,1583.000000,1666.000000,3199.000000,2739.500000,...,136.979996,30.320000,111.339996,70.320000,14.750000,104.650002,137.100006,133.460007,30.930000,174.245773
2023-04-18,254.0,1471.000000,2487.000000,4225.000000,1030.000000,768.000000,1587.000000,1672.000000,3200.000000,2740.000000,...,139.589996,30.160000,112.150002,69.760002,14.730000,104.190002,137.580002,134.320007,30.459999,174.076141
2023-04-19,253.0,1485.000000,2482.000000,4255.000000,1026.000000,766.000000,1584.000000,1676.000000,3203.000000,2744.000000,...,139.539993,30.080000,112.120003,70.589996,14.260000,103.739998,137.119995,138.119995,32.720001,174.604996


,1557.T,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,1925.T,1928.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-01,29030.0,1181.539185,2578.757080,5039.168457,1047.106689,984.559326,1365.645874,1761.031006,3409.902344,1658.666992,...,131.141342,19.050804,181.230804,40.532646,25.731714,66.262154,79.292213,109.272720,47.827251,80.757622
2018-05-02,29060.0,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,129.728363,19.190237,180.456192,40.437065,23.412752,64.859131,73.398384,109.749229,47.948879,77.368500
2018-05-03,29060.0,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,127.625664,19.153540,181.013535,40.384930,22.560074,65.565346,75.658890,107.618919,47.305962,77.833290
2018-05-04,29060.0,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,128.937729,19.219591,181.759827,40.306732,22.615856,65.989082,75.439262,108.590645,48.183456,78.975906
2018-05-07,29160.0,1162.568604,2546.771240,5039.168457,1042.035889,967.918762,1354.645508,1720.817017,3436.165283,1651.217651,...,129.021820,19.278297,181.287476,40.080833,22.679607,65.932564,76.061584,109.310104,48.226902,79.547218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-17,55370.0,1503.000000,2478.000000,4245.000000,1030.000000,780.000000,1583.000000,1666.000000,3199.000000,2739.500000,...,136.979996,30.320000,111.339996,70.320000,14.750000,104.650002,137.100006,133.460007,30.930000,174.245773
2023-04-18,55640.0,1471.000000,2487.000000,4225.000000,1030.000000,768.000000,1587.000000,1672.000000,3200.000000,2740.000000,...,139.589996,30.160000,112.150002,69.760002,14.730000,104.190002,137.580002,134.320007,30.459999,174.076141
2023-04-19,55630.0,1485.000000,2482.000000,4255.000000,1026.000000,766.000000,1584.000000,1676.000000,3203.000000,2744.000000,...,139.539993,30.080000,112.120003,70.589996,14.260000,103.739998,137.119995,138.119995,32.720001,174.604996


,8789.T,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,1925.T,1928.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-01,93.0,1181.539185,2578.757080,5039.168457,1047.106689,984.559326,1365.645874,1761.031006,3409.902344,1658.666992,...,131.141342,19.050804,181.230804,40.532646,25.731714,66.262154,79.292213,109.272720,47.827251,80.757622
2018-05-02,92.0,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,129.728363,19.190237,180.456192,40.437065,23.412752,64.859131,73.398384,109.749229,47.948879,77.368500
2018-05-03,92.0,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,127.625664,19.153540,181.013535,40.384930,22.560074,65.565346,75.658890,107.618919,47.305962,77.833290
2018-05-04,92.0,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,128.937729,19.219591,181.759827,40.306732,22.615856,65.989082,75.439262,108.590645,48.183456,78.975906
2018-05-07,91.0,1162.568604,2546.771240,5039.168457,1042.035889,967.918762,1354.645508,1720.817017,3436.165283,1651.217651,...,129.021820,19.278297,181.287476,40.080833,22.679607,65.932564,76.061584,109.310104,48.226902,79.547218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-17,65.0,1503.000000,2478.000000,4245.000000,1030.000000,780.000000,1583.000000,1666.000000,3199.000000,2739.500000,...,136.979996,30.320000,111.339996,70.320000,14.750000,104.650002,137.100006,133.460007,30.930000,174.245773
2023-04-18,67.0,1471.000000,2487.000000,4225.000000,1030.000000,768.000000,1587.000000,1672.000000,3200.000000,2740.000000,...,139.589996,30.160000,112.150002,69.760002,14.730000,104.190002,137.580002,134.320007,30.459999,174.076141
2023-04-19,67.0,1485.000000,2482.000000,4255.000000,1026.000000,766.000000,1584.000000,1676.000000,3203.000000,2744.000000,...,139.539993,30.080000,112.120003,70.589996,14.260000,103.739998,137.119995,138.119995,32.720001,174.604996


,1893.T,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,1925.T,1928.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-01,733.416992,1181.539185,2578.757080,5039.168457,1047.106689,984.559326,1365.645874,1761.031006,3409.902344,1658.666992,...,131.141342,19.050804,181.230804,40.532646,25.731714,66.262154,79.292213,109.272720,47.827251,80.757622
2018-05-02,737.685974,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,129.728363,19.190237,180.456192,40.437065,23.412752,64.859131,73.398384,109.749229,47.948879,77.368500
2018-05-03,737.685974,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,127.625664,19.153540,181.013535,40.384930,22.560074,65.565346,75.658890,107.618919,47.305962,77.833290
2018-05-04,737.685974,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,128.937729,19.219591,181.759827,40.306732,22.615856,65.989082,75.439262,108.590645,48.183456,78.975906
2018-05-07,741.101196,1162.568604,2546.771240,5039.168457,1042.035889,967.918762,1354.645508,1720.817017,3436.165283,1651.217651,...,129.021820,19.278297,181.287476,40.080833,22.679607,65.932564,76.061584,109.310104,48.226902,79.547218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-17,631.000000,1503.000000,2478.000000,4245.000000,1030.000000,780.000000,1583.000000,1666.000000,3199.000000,2739.500000,...,136.979996,30.320000,111.339996,70.320000,14.750000,104.650002,137.100006,133.460007,30.930000,174.245773
2023-04-18,635.000000,1471.000000,2487.000000,4225.000000,1030.000000,768.000000,1587.000000,1672.000000,3200.000000,2740.000000,...,139.589996,30.160000,112.150002,69.760002,14.730000,104.190002,137.580002,134.320007,30.459999,174.076141
2023-04-19,629.000000,1485.000000,2482.000000,4255.000000,1026.000000,766.000000,1584.000000,1676.000000,3203.000000,2744.000000,...,139.539993,30.080000,112.120003,70.589996,14.260000,103.739998,137.119995,138.119995,32.720001,174.604996


,MSFT,1605.T,1721.T,1801.T,1802.T,1803.T,1808.T,1812.T,1925.T,1928.T,...,WHR,WMB,WYNN,XEL,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-01,89.658585,1181.539185,2578.757080,5039.168457,1047.106689,984.559326,1365.645874,1761.031006,3409.902344,1658.666992,...,131.141342,19.050804,181.230804,40.532646,25.731714,66.262154,79.292213,109.272720,47.827251,80.757622
2018-05-02,88.252373,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,129.728363,19.190237,180.456192,40.437065,23.412752,64.859131,73.398384,109.749229,47.948879,77.368500
2018-05-03,88.780869,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,127.625664,19.153540,181.013535,40.384930,22.560074,65.565346,75.658890,107.618919,47.305962,77.833290
2018-05-04,89.809593,1152.671021,2552.822754,5030.642090,1058.093140,982.710327,1367.217529,1739.248291,3400.583008,1648.321045,...,128.937729,19.219591,181.759827,40.306732,22.615856,65.989082,75.439262,108.590645,48.183456,78.975906
2018-05-07,90.810013,1162.568604,2546.771240,5039.168457,1042.035889,967.918762,1354.645508,1720.817017,3436.165283,1651.217651,...,129.021820,19.278297,181.287476,40.080833,22.679607,65.932564,76.061584,109.310104,48.226902,79.547218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-17,288.799988,1503.000000,2478.000000,4245.000000,1030.000000,780.000000,1583.000000,1666.000000,3199.000000,2739.500000,...,136.979996,30.320000,111.339996,70.320000,14.750000,104.650002,137.100006,133.460007,30.930000,174.245773
2023-04-18,288.369995,1471.000000,2487.000000,4225.000000,1030.000000,768.000000,1587.000000,1672.000000,3200.000000,2740.000000,...,139.589996,30.160000,112.150002,69.760002,14.730000,104.190002,137.580002,134.320007,30.459999,174.076141
2023-04-19,288.450012,1485.000000,2482.000000,4255.000000,1026.000000,766.000000,1584.000000,1676.000000,3203.000000,2744.000000,...,139.539993,30.080000,112.120003,70.589996,14.260000,103.739998,137.119995,138.119995,32.720001,174.604996
